# **LIBRERIAS**

### Directorio

In [1]:
import os               # Gestion de directorios
import requests         # Interactuar con rutas de archivos

### Tratamiento de datos

In [2]:
import pandas as pd       # Tratamiento de datos
import numpy as np        # Operaciones y arreglos multidimencionales
from io import BytesIO    # Tratamiento de datos en bytes

### Algoritmos predictivos

In [3]:
# Entrenamiento
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# convertir una colección de documentos de texto en una matriz de recuentos de tokens.
from sklearn.feature_extraction.text import CountVectorizer

# Clasificación de texto
from sklearn.naive_bayes import MultinomialNB

# Métrica de evaluación para clasificación
from sklearn.metrics import accuracy_score

# Support Vector Machine
from sklearn.svm import SVC

In [4]:
# Exportacion del modelo
import joblib

# **ARCHIVOS DE GITHUB**

## **Links**

In [5]:
# Directorios de github
directorio = 'https://raw.githubusercontent.com/JosueAVD27/Predicciones-de-Genero/main/'
directorio_data = directorio + 'Data/'

In [6]:
# Matrizes de datos
matriz_genero = directorio_data + "GENERO_NOMBRES.xlsx"

# Datos de prueba
matriz_genero_test = directorio_data + 'PREDICCION_GENERO.xlsx'

# Modelos entrenados
model_genre = directorio + 'GENERO_model.pkl';

## **Obtener matrizes**

### **Matriz de datos**

#### Generos

In [7]:
# Realizar una solicitud GET al archivo de Excel en el repositorio
response_generos = requests.get(matriz_genero)

# Verificar el estado de la respuesta
if response_generos.status_code == 200:

    # La solicitud fue exitosa, leer el contenido del archivo Excel en un DataFrame de pandas
    excel_genero = response_generos.content
    df_genero = pd.read_excel(BytesIO(excel_genero))    # ▬▬▬

else:
    # La solicitud falló, imprimir el código de estado de la respuesta
    print("Error al obtener el archivo de Excel:", response_generos.status_code)

### **Datos de prueba**

#### Genero

In [8]:
# Realizar una solicitud GET al archivo de Excel en el repositorio
response_generos_test = requests.get(matriz_genero_test)

# Verificar el estado de la respuesta
if response_generos_test.status_code == 200:

    # La solicitud fue exitosa, leer el contenido del archivo Excel en un DataFrame de pandas
    excel_genero_test = response_generos_test.content
    df_genero_test = pd.read_excel(BytesIO(excel_genero_test))    # ▬▬▬

else:
    # La solicitud falló, imprimir el código de estado de la respuesta
    print("Error al obtener el archivo de Excel:", response_generos_test.status_code)

# **1. Tratamiento y limpieza de datos**

## 1.1. **Datos**

#### 1.1.1. Matriz de datos

In [9]:
# Obtener matriz de generos
df_genero.head()

,NOMBRE,GENERO
0,"TAPIA CERVANTES, CARLOS EFREN",H
1,"RAMIREZ BOSQUEZ , JOSHUA RIC",H
2,"BARRAGAN INTRIAGO , JONATHAN",H
3,"MONTESDEOCA POSSO, ERICK ISMAE",H
4,"ROCOHANO RAMOS, RONNY GONZALO",H


In [10]:
len(df_genero)

1584

#### 1.1.2. Datos test

In [11]:
# Obtener matriz de generos
df_genero_test.head()

,NOMBRE,GENERO
0,"ORDOÑEZ SOLIS, ANDRÉS SANTIAGO",NaN
1,"VALLEJO ARTEAGA, ELENA CAROLINA",NaN
2,"GUERRERO LOZADA, MARCO ANTONIO",NaN
3,"BRAVO ESPINOZA, DANIELA PAOLA",NaN
4,"SÁNCHEZ MENDOZA, FABIAN ALEJANDRO",NaN


## 1.2. **Predicción Genero**

### 1.2.1. **Entrenamiento**

#### Naive Bayes multinomial

In [12]:
# Naive Bayes multinomial
dataframe_genero = df_genero.copy()

# Preprocesamiento de datos
dataframe_genero[['Apellidos', 'Nombres']] = dataframe_genero['NOMBRE'].str.split(pat=',', n=1, expand=True)
dataframe_genero['GENERO'] = dataframe_genero['GENERO'].map({'H': 1, 'M': 0})

# Agregar variantes de nombres
dataframe_genero['Nombres'] = dataframe_genero['Nombres'] + ' ' + dataframe_genero['Apellidos']
dataframe_genero['Nombres_invertidos'] = dataframe_genero['Apellidos'] + ' ' + dataframe_genero['Nombres']

# Unir los nombres y las variantes
nombres = dataframe_genero['Nombres'] + ' ' + dataframe_genero['Nombres_invertidos']

# Extracción de características
vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 3))
X = vectorizer.fit_transform(nombres)
y = dataframe_genero['GENERO']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
model = MultinomialNB()
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

# Evaluación del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo: {:.2f}%".format(accuracy * 100))

Precisión del modelo: 91.48%


In [13]:
# Guardar el modelo entrenado y el CountVectorizer en un solo archivo pickle
joblib.dump((model, vectorizer), './modelo_genero.pkl')

['./modelo_genero.pkl']

#### ERROR SVM no paso los test de prueba

In [14]:
from sklearn.impute import SimpleImputer

dataframe_genero = df_genero.copy()

# Preprocesamiento de datos
dataframe_genero[['Apellidos', 'Nombres']] = dataframe_genero['NOMBRE'].str.split(pat=',', n=1, expand=True)
dataframe_genero['GENERO'] = dataframe_genero['GENERO'].map({'H': 1, 'M': 0})

# Agregar variantes de nombres
dataframe_genero['Nombres'] = dataframe_genero['Nombres'] + ' ' + dataframe_genero['Apellidos']
dataframe_genero['Nombres_invertidos'] = dataframe_genero['Apellidos'] + ' ' + dataframe_genero['Nombres']

# Unir los nombres y las variantes
nombres = dataframe_genero['Nombres'] + ' ' + dataframe_genero['Nombres_invertidos']

# Extracción de características
svm_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 3))
X = svm_vectorizer.fit_transform(nombres)
y = dataframe_genero['GENERO']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Manejar los valores faltantes en y_train
imputer = SimpleImputer(strategy='most_frequent')
y_train_imputed = imputer.fit_transform(y_train.values.reshape(-1, 1)).ravel()

# Entrenamiento del modelo SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train_imputed)

# Predicción
y_pred = svm_model.predict(X_test)

# Evaluación del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo SVM: {:.2f}%".format(accuracy * 100))


Precisión del modelo SVM: 90.22%


### 1.2.2. **Pruebas Naive Bayes multinomial**

In [15]:
# Ejemplo de predicción
nuevo_nombre = ['Velasquez Mora, MARON OMAR']

# Preprocesamiento del nuevo nombre
nuevo_nombre_df = pd.DataFrame({'NOMBRE': nuevo_nombre})
nuevo_nombre_df[['Apellidos', 'Nombres']] = nuevo_nombre_df['NOMBRE'].str.split(pat=',', n=1, expand=True)

# Extracción de características del nuevo nombre
X_nuevo = vectorizer.transform(nuevo_nombre_df['Nombres'])

# Hacer la predicción
prediccion = model.predict(X_nuevo)

# Imprimir la predicción
genero_predicho = 'Hombre' if prediccion[0] == 1 else 'Mujer'
print(f"El nombre '{nuevo_nombre[0]}' se predice como {genero_predicho}.")

El nombre 'Velasquez Mora, MARON OMAR' se predice como Hombre.


In [16]:
# Obtener matriz de generos
df_nuevo_genero_test = df_genero_test.copy()

# Preprocesar los nombres en la matriz de prueba
df_nuevo_genero_test[['Apellidos', 'Nombres']] = df_nuevo_genero_test['NOMBRE'].str.split(pat=',', n=1, expand=True)

# Extraer características de los nombres
X_test = vectorizer.transform(df_nuevo_genero_test['Nombres'])

# Realizar predicciones utilizando el modelo entrenado
y_pred_test = model.predict(X_test)

# Asignar las predicciones de género a la columna 'GENERO' en la matriz de prueba
df_nuevo_genero_test['GENERO'] = ['H' if pred == 1 else 'M' for pred in y_pred_test]

In [17]:
# Mostrar la matriz con las predicciones de género
df_nuevo_genero_test

,NOMBRE,GENERO,Apellidos,Nombres
0,"ORDOÑEZ SOLIS, ANDRÉS SANTIAGO",H,ORDOÑEZ SOLIS,ANDRÉS SANTIAGO
1,"VALLEJO ARTEAGA, ELENA CAROLINA",M,VALLEJO ARTEAGA,ELENA CAROLINA
2,"GUERRERO LOZADA, MARCO ANTONIO",H,GUERRERO LOZADA,MARCO ANTONIO
3,"BRAVO ESPINOZA, DANIELA PAOLA",M,BRAVO ESPINOZA,DANIELA PAOLA
4,"SÁNCHEZ MENDOZA, FABIAN ALEJANDRO",H,SÁNCHEZ MENDOZA,FABIAN ALEJANDRO
5,"RAMIREZ VILLACIS, SOFIA ALEJANDRA",M,RAMIREZ VILLACIS,SOFIA ALEJANDRA
6,"MORENO AGUILAR, JORGE LUIS",H,MORENO AGUILAR,JORGE LUIS
7,"JIMENEZ CARRIÓN, CLAUDIA PATRICIA",M,JIMENEZ CARRIÓN,CLAUDIA PATRICIA
8,"VERA ZAMBRANO, CARLOS ANDRÉS",H,VERA ZAMBRANO,CARLOS ANDRÉS
9,"TORRES PONCE, LUCIA MARGARITA",M,TORRES PONCE,LUCIA MARGARITA


In [18]:
# Guardar el nuevo DataFrame en un nuevo archivo Excel
df_nuevo_genero_test.to_excel(f"./TABLA_GENEROS_PREDECIDOS.xlsx", index=False)

# **3. Pruebas del modelo entrenado**

In [20]:
# Realizar una solicitud GET al archivo del modelo en el repositorio
response_modelo = requests.get(model_genre)

# Verificar el estado de la respuesta
if response_modelo.status_code == 200:
    # La solicitud fue exitosa, cargar el modelo desde el contenido descargado
    loaded_model, loaded_vectorizer = joblib.load(BytesIO(response_modelo.content))
    print("Modelo cargado exitosamente.")
else:
    # La solicitud falló, imprimir el código de estado de la respuesta
    print("Error al obtener el modelo:", response_modelo.status_code)

Modelo cargado exitosamente.


In [21]:
# Obtener matriz de generos
df_nuevo_genero_test = df_genero_test.copy()

# Extracción de características utilizando el CountVectorizer cargado
X_nuevo = loaded_vectorizer.transform(df_nuevo_genero_test['NOMBRE'])

# Realizar predicciones usando el modelo cargado
predicciones_nuevo = loaded_model.predict(X_nuevo)

# Asignar las predicciones a la columna 'GENERO' de la nueva matriz
df_nuevo_genero_test['GENERO'] = predicciones_nuevo

# Mapear las predicciones de nuevo a 'H' para hombre y 'M' para mujer
df_nuevo_genero_test['GENERO'] = df_nuevo_genero_test['GENERO'].map({1: 'H', 0: 'M'})

# Imprimir la nueva matriz con las predicciones de género
print(df_nuevo_genero_test)


                              NOMBRE GENERO
0     ORDOÑEZ SOLIS, ANDRÉS SANTIAGO      H
1    VALLEJO ARTEAGA, ELENA CAROLINA      M
2     GUERRERO LOZADA, MARCO ANTONIO      H
3      BRAVO ESPINOZA, DANIELA PAOLA      M
4  SÁNCHEZ MENDOZA, FABIAN ALEJANDRO      H
5  RAMIREZ VILLACIS, SOFIA ALEJANDRA      M
6         MORENO AGUILAR, JORGE LUIS      H
7  JIMENEZ CARRIÓN, CLAUDIA PATRICIA      M
8       VERA ZAMBRANO, CARLOS ANDRÉS      H
9      TORRES PONCE, LUCIA MARGARITA      M


In [22]:
# Obtener matriz de generos
# df_nuevo_genero_test_uno = ['CASTILLO ZAMBRANO, GEMA GABRIE']
df_nuevo_genero_test_uno = ['CASTILLO ZAMBRANO, MARON OMAR']

# X_nuevo_uno = loaded_vectorizer.transform(['MARON OMAR'])
X_nuevo_uno = loaded_vectorizer.transform(df_nuevo_genero_test_uno)

predicciones_nuevo_uno = loaded_model.predict(X_nuevo_uno)

# Imprimir la predicción
genero_predicho_uno = 'Hombre' if predicciones_nuevo_uno[0] == 1 else 'Mujer'
print(f"El nombre '{df_nuevo_genero_test_uno[0]}' se predice como {genero_predicho_uno}.")

El nombre 'CASTILLO ZAMBRANO, MARON OMAR' se predice como Hombre.
